# Ledger - World Cup Betting Demo - SQL Server 2022

## ![Database Administrator](../../../../../media/features/ledger/Pieter.jpg "Database Administrator") Updatable Ledger Tables


All the soccer games are stored in an updatable ledger table. Every update on the games or the odds needs to verifiable.

In [ ]:
SELECT	t.[commit_time] AS [CommitTime] 
	, t.[principal_name] AS [UserName]
	, l.[MoneyLineID]
    , l.[HomeCountry]
	, l.[HomeCountryOdds]
	, l.[DrawOdds]
	, l.[VisitCountry]
	, l.[VisitCountryOdds]
    , l.[GameDateTime]
	, l.[ledger_operation_type_desc] AS Operation
	FROM [dbo].[MoneyLine_Ledger] l
	JOIN sys.database_ledger_transactions t
	ON t.transaction_id = l.ledger_transaction_id
	ORDER BY t.commit_time DESC;

In [ ]:
UPDATE MoneyLine
SET [HomeCountryOdds]=500
WHERE MoneyLineID=1

In [ ]:
SELECT
	t.[commit_time] AS [CommitTime] 
	, t.[principal_name] AS [UserName]
	, l.[MoneyLineID]
    , l.[HomeCountry]
	, l.[HomeCountryOdds]
	, l.[DrawOdds]
	, l.[VisitCountry]
	, l.[VisitCountryOdds]
    , l.[GameDateTime]
	, l.[ledger_operation_type_desc] AS Operation
	FROM [dbo].[MoneyLine_Ledger] l
	JOIN sys.database_ledger_transactions t
	ON t.transaction_id = l.ledger_transaction_id
	WHERE t.[commit_time] > DATEADD(MINUTE, -10, GETDATE())
	ORDER BY t.commit_time DESC;

## Append-Only Ledger Table Bets

## ![Auditor](../../../../../media/features/ledger/Michael.jpg "Auditor") The auditor can verify the database to be sure that the data can be trusted

<span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px; background-color: rgb(255, 255, 255);">Michael, who is an auditor, performs a routine review of changes in the Worldcup database. As his first step, Michael runs the ledger verification to make sure he can trust the data he’s going to examine. The result should be "Success"!</span>

In [ ]:
DECLARE @digest_locations NVARCHAR(MAX) = (SELECT * FROM sys.database_ledger_digest_locations FOR JSON AUTO, INCLUDE_NULL_VALUES);
    SELECT @digest_locations as digest_locations;
    BEGIN TRY
        EXEC sys.sp_verify_database_ledger_from_digest_storage @digest_locations;
    SELECT 'Ledger verification succeeded.' AS Result;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH

## ![Database Administrator](../../../../../media/features/ledger/Pieter.jpg "Database Administrator") Let's try to modify the bet

<span style="font-family: Calibri, sans-serif; font-size: 11pt;">Pieter, the malicious DBA tries to manipulate his bet but noticed that it’s an append-only ledger table and that data cannot be modified.&nbsp;</span>

In [ ]:
USE WorldCup
GO
SELECT * from Bets
WHERE FirstName='Pieter' and Lastname='Vanhove'

In [ ]:
UPDATE Bets
SET Payout=-10000
WHERE BetID=6

## ![Database Administrator](../../../../../media/features/ledger/Pieter.jpg "Database Administrator") Let's tamper the data

Pieter thinks he's smart and tampers with the data directly into the data file by using a stored procedure.

In [ ]:
EXECUTE sp_TamperWithBet
	@ID=6,
	@PayOut=-10000

## ![Auditor](../../../../../media/features/ledger/Michael.jpg "Auditor") Verify the database again

<span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px; background-color: rgb(255, 255, 255);">Michael, reviews the Worldcup database again after the tournament is over.&nbsp;</span>

In [3]:
DECLARE @digest_locations NVARCHAR(MAX) = (SELECT * FROM sys.database_ledger_digest_locations FOR JSON AUTO, INCLUDE_NULL_VALUES);
    
    BEGIN TRY
        EXEC sys.sp_verify_database_ledger_from_digest_storage @digest_locations;
    SELECT 'Ledger verification succeeded.' AS Result;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH

: Msg 37371, Level 16, State 1, Procedure sys.sp_verify_database_ledger_from_digest_storage, Line 1
The computed hash from 'Bets' and the associated history table does not match the hash persisted in sys.database_ledger_transactions for transaction 1004.

: Msg 37392, Level 16, State 1, Procedure sys.sp_verify_database_ledger_from_digest_storage, Line 1
Ledger verification failed.

Total execution time: 00:00:00.585

## Review the history of the transaction

In [4]:
SELECT
  t.[commit_time] AS [CommitTime] 
    ,l.[MoneylineID]
    ,l.[FirstName]
    ,l.[LastName]
    ,l.[Country]
    ,l.[Bet]
    ,l.[Payout]
    ,l.[BetDateTime]
	, l.[ledger_operation_type_desc] AS Operation
	FROM [dbo].[Bets_Ledger] l
	JOIN [sys].[database_ledger_transactions] t
	ON t.[transaction_id] = l.[ledger_transaction_id]
	WHERE t.transaction_id=1004;

(1 row affected)

Total execution time: 00:00:00.042

CommitTime,MoneylineID,FirstName,LastName,Country,Bet,Payout,BetDateTime,Operation
2022-12-01 16:21:50.7400000,12,Pieter,Vanhove,Belgium,300.00,-10000.00,2022-12-01 16:21:50.7400000,INSERT
